<a href="https://colab.research.google.com/github/straus91/dicom-jpr2022/blob/master/DICOM_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DICOM

<br><br>
## Pydicom
### Neste curso iremos trabalhar com uma biblioteca chamada pydicom, que possui muiotas funções já preparadas para trabalhar com esse formato


In [ ]:
!pip install pydicom
import pydicom

!pip install scikit-image
!pip install opencv-python
!pip install matplotlib
from pydicom.pixel_data_handlers import util
from matplotlib import pyplot
import numpy as np
import cv2
import skimage
from skimage import io
from skimage import filters
import os
import pandas as pd

!git clone https://github.com/straus91/dicom-jpr2022.git

### O primeiro passo será carregar um arquivo dicom, entao precisamos:
1. localizar o arquivo DICOM
2. carregar o aqruivo DICOM para uma variável.


In [ ]:
meu_dicom = pydicom.dcmread("dicom-jpr2022/dicom/exemplo_1.dcm")

<img src ="notebook_images/funcao_3.png">

In [ ]:
meu_dicom

### Acabamos de carregar todas as informações do arquivo DICOM para dentro da variavel chamada meu_dicom
### Conseguimos agora ver todos DataElements e os seus respectivos valores
### Vamos explorar o DICOM um pouco

In [ ]:
meu_dicom.StudyDescription

In [ ]:
meu_dicom[0x00081030]

In [ ]:
meu_dicom.dir()

In [ ]:
if "PixelData" in meu_dicom:
    print("Esse exame tem PixelData")
else:
    print("Esses exame NÃO tem PixelData")

In [ ]:
meu_dicom.PixelData

### Para conseguir mostrar a imagem dentro do PixelData de forma adequada é preciso processa-la, para isso vamos usar alguma ferramentadas do módulo pyplot da bilbioteca matplotlib
### Vamos usar especificamente a função show() e vamos passar como input o próprio PixelData

In [ ]:
pyplot.imshow(meu_dicom.pixel_array, cmap=pyplot.cm.gray)
pyplot.show()

### Agora podemos até salvar apenas a imagem, sem nenhum outro DataElement em um outro formato, como por exemplo png
### Para isso vamos usar a biblioteca opencv2 e numpy

In [ ]:


#Carregar o PixelArray
arr = meu_dicom.pixel_array

#Aplicar lookuptable
arr = util.apply_modality_lut(arr, meu_dicom)

# Normalizar o array
arr = arr-np.min(arr)
arr = arr/np.max(arr)
arr = (arr*255).astype(np.uint8)
cv2.imwrite("rm.jpg", arr)

### Podemos também modificar a imagem antes de salva-la, para isso vamos usar ferramentas de uma outra biblioteca chamda scikit-image

In [ ]:
imagem = io.imread("rm.jpg", as_gray = True)

In [ ]:
imagem_nova = filters.sobel(imagem)
pyplot.imshow(imagem_nova, cmap=pyplot.cm.gray)
pyplot.show()

In [ ]:
imagem_nova2 = filters.apply_hysteresis_threshold(imagem_nova, 0.09, 0.1)
pyplot.imshow(imagem_nova2, cmap=pyplot.cm.gray)
pyplot.show()

In [ ]:
imagem_nova = filters.prewitt_v(imagem)
pyplot.imshow(imagem_nova, cmap=pyplot.cm.gray)
pyplot.show()

### Vamos agora fazer algumas operações com mais de um arquivo e tentar agregar dados que possam ser uteis na prática

In [ ]:
lista_arquivos = os.listdir("dicom-jpr2022/dicom")

In [ ]:
lista_arquivos

In [ ]:
#criamos uma lista vazia
lista_dados = []

for elemento in lista_arquivos:
    
    #Carregaremos todos os arquivos da lista
    caminho_arquivo = "dicom-jpr2022/dicom/" + elemento
    dicom = pydicom.dcmread(caminho_arquivo)
    print("carregando arquivo:", caminho_arquivo)
    
    # Vamos salvar algun dados desse para em variaveis
    sexo = dicom.PatientSex
    modelo = dicom.ManufacturerModelName
    idade = dicom.PatientAge
    serie = dicom.SeriesDescription
    
    #Vamos criar uma lista e incluir todos esses dados na lista, sempre na mesma ordem
    dados_exame = [sexo, modelo, idade, serie]
    
    #Criamos aqui uma lista de lista (adicionamos uma lista a outra lista)
    lista_dados.append(dados_exame)
    print("carregado com sucesso")
    print("")

    
    
    

In [ ]:
lista_dados

In [ ]:
#Aqui estamos usando uma biblioteca chamada pandas, que é util para manipular dados tabulares
df = pd.DataFrame(lista_dados, columns=["sexo", "modelo", "idade", "serie"])

In [ ]:
df

### Modificando e salvando um DICOM

### Para finalizar vamos editar um dicom e salva-lo

In [ ]:
meu_dicom = pydicom.dcmread("dicom-jpr2022/dicom/exemplo_1.dcm")

In [ ]:
meu_dicom.PatientName

In [ ]:
meu_dicom.PatientName = "Paciente 01"

In [ ]:
meu_dicom.save_as("dicom_modificado.dcm")

In [ ]:
dicom_original = pydicom.dcmread("dicom-jpr2022/dicom/exemplo_1.dcm")
print("Nome do paciente no dicom original:", dicom_original.PatientName)
dicom_alterado = pydicom.dcmread("dicom_modificado.dcm")
print("Nome do paciente no dicom original:", dicom_alterado.PatientName)